In [7]:
from funcs import load_matrices, matrix_funcs
import numpy as np
import matplotlib.pyplot as plt
from scipy.special import airy, airye, spherical_jn as sph_jn, spherical_yn as sph_yn
import os

dir = 'H_0_plus_p'

r, hls, ddrls = load_matrices(dir)
emin = hls[-1,0,0]
n = len(hls[0])
shift = np.array([emin*np.eye(len(hls[0]), dtype=complex)]*len(r))
vd = (hls-shift)/4.637
hfunc, ddrfunc = matrix_funcs(r, vd, ddrls)
emax = np.abs(np.array(hfunc(r[0])[-1,-1]))
w = lambda rs, l, e: np.array([e*np.eye(n)-(hfunc(r)+l*(l*1)/(r**2)*np.eye(n)) for r in rs])
wp = lambda rs, l: np.array([-(hfunc(r,1)-2*l*(l*1)/(r**3)*np.eye(n)) for r in rs])
es = np.array([1])
for i in range(n):
    emax = np.abs(np.array(hfunc(r[0])[i,i]))
    emin = np.abs(np.array(hfunc(r[-1])[i,i]))
    es = np.concatenate((es, np.power(10, np.arange(np.log10(emin if emin!=0 else 1), np.log10(emax), 0.01))))
es = np.unique(np.round(np.sort(es)))
sigmas = np.zeros((len(es), n, n))
ecm = es*4.637+np.real(hls[-1,0,0])

In [14]:
ei = 270
e = es[ei]
lmax = int(np.round(np.sqrt(e)*np.max(r[-1]))*0.8)
ks = np.diag(np.sqrt(e*np.eye(n)-hfunc(r[-1])))
yf = np.zeros((lmax, n, n), dtype=complex)
nt = np.sum(np.diag(vd[-1]) < e)

In [15]:
for l in range(lmax):
    wtest = w(r[:-1]+np.diff(r)/2, l, e)
    wptest = wp(r[:-1]+np.diff(r)/2, l)
    k2 = np.zeros((len(wtest), n))
    t = np.zeros((len(wtest), n, n), dtype=complex)
    wpt = np.zeros((len(wtest), n, n), dtype=complex)
    for i in range(len(wtest)):
        k2[i], t[i] = np.linalg.eigh(wtest[i])
        wpt[i] = np.matrix(t[i]).H @ (np.matrix(wptest[i]) @ np.matrix(t[i]))
    p = np.array([np.matrix(t[i]).H @ np.matrix(t[i-1]) if i > 0 else np.matrix(t[0]) for i in range(len(wtest))])
    c1 = np.zeros((len(wtest), n, n), dtype=complex)
    c2 = np.zeros((len(wtest), n, n), dtype=complex)
    c3 = np.zeros((len(wtest), n, n), dtype=complex)
    c4 = np.zeros((len(wtest), n, n), dtype=complex)
    for i in range(len(wtest)):
        phase = 2*np.pi/3
        alpha = np.array([-(wpt[i,j,j])**(1./3.) if np.allclose(-(wpt[i,j,j])**(1./3.), np.real(-(wpt[i,j,j])**(1./3.)))
                        else -(wpt[i,j,j])**(1./3.)*np.exp(-1j*np.angle(-(wpt[i,j,j])**(1./3.))) for j in range(n)])
        beta = k2[i]/np.diag(wpt[i])
        wr = alpha/np.pi
        rhob = np.diff(r)[i]/2
        x = np.array([[alpha[j]*(-rhob+beta[j]), alpha[j]*(rhob+beta[j])] for j in range(n)])
        for j in range(n):
            if np.any(np.real(x[j]) > 100):
                dksi = np.diff((2./3)*(x[j]**(3./2)))[0]
                aie, aiep, bie, biep = airye(x[j])
                c1[i,j,j] = np.pi*(np.exp(-dksi)*aie[1]*biep[0]-np.exp(dksi)*bie[1]*aiep[0]) #/wr[j]
                c2[i,j,j] = np.pi*(np.exp(dksi)*bie[1]*aie[0]-np.exp(-dksi)*aie[1]*bie[0])/alpha[j]
                c3[i,j,j] = np.pi*alpha[j]*(np.exp(-dksi)*aiep[1]*biep[0]-np.exp(dksi)*biep[1]*aiep[0]) #/wr[j]
                c4[i,j,j] = np.pi*(np.exp(dksi)*biep[1]*aie[0]-np.exp(-dksi)*aiep[1]*bie[0]) #/wr[j]
            else:
                ai, aip, bi, bip = airy(x[j])
                c1[i,j,j] = np.pi*(ai[1]*bip[0]-bi[1]*aip[0]) #/wr[j]
                c2[i,j,j] = np.pi*(bi[1]*ai[0]-ai[1]*bi[0])/alpha[j]
                c3[i,j,j] = np.pi*alpha[j]*(aip[1]*bip[0]-bip[1]*aip[0]) #/wr[j]
                c4[i,j,j] = np.pi*(bip[1]*ai[0]-aip[1]*bi[0]) #/wr[j]
    y = np.zeros((len(r), n, n), dtype=complex)
    y[0] = 1e308*np.eye(n, dtype=complex)
    for i in range(len(wtest)):
        prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
        y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)
        # try:
        #     prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
        # except np.linalg.LinAlgError:
        #     print(np.matrix(c1[i] + c2[i] @ y[i]))
        # print(prey)
        # y[i+1] = np.matrix(p[i]) @ (prey @ np.matrix(p[i]).H)
        # print(y[i+1])
    yf[l] = np.matrix(t[-1]).H @ (y[-1] @ np.matrix(t[-1]))
jmat = np.array([np.diag([sph_jn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
nmat = np.array([np.diag([sph_yn(int(l), k*r[-1]) for k in ks]) for l in range(lmax)])
jpmat = np.array([np.diag([k*sph_jn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
npmat = np.array([np.diag([k*sph_yn(int(l), k*r[-1], derivative=True) for k in ks]) for l in range(lmax)])
rmat = np.array([-np.matrix(npmat[l] - yf[l] @ nmat[l]).I @ np.matrix(jpmat[l] - yf[l] @ jmat[l]) for l in range(lmax)])
smat = np.array([np.matrix(np.eye(nt) + 1j*rmat[l,:nt,:nt]).I @ np.matrix(np.eye(nt) - 1j*rmat[l,:nt,:nt]) if not np.all(np.isnan(rmat[l,:nt,:nt])) else np.eye(nt) for l in range(lmax)])
sigmas[ei,:nt,:nt] = np.pi * 1e-16 * np.sum(np.array([(2*l+1)*(np.abs(smat[l]-np.eye(nt))**2) for l in range(lmax)]),axis=0) @ np.matrix(np.abs(np.diag(ks[:nt]))**2).I
print(ecm[ei])
print(sigmas[ei])

/tmp/ipykernel_138081/4288112464.py:40: RuntimeWarning: overflow encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)
/tmp/ipykernel_138081/4288112464.py:40: RuntimeWarning: invalid value encountered in matmul
  prey = np.matrix(c3[i] + c4[i] @ y[i]) @ (np.matrix(c1[i] + c2[i] @ y[i]).I)


108588.22499999999
[[1.67992657e-14 9.11438963e-16 1.83162619e-17 1.88406616e-18
  7.60637429e-18]
 [6.16711969e-16 3.92496014e-14 8.92368026e-17 1.11949567e-17
  3.44531401e-18]
 [5.82151236e-18 4.18585880e-17 2.87871450e-14 1.21238224e-15
  7.20703646e-20]
 [5.06378485e-19 4.44419146e-18 1.02862340e-15 2.98341708e-14
  3.74656450e-20]
 [1.46294961e-20 9.81660409e-21 4.36235176e-22 2.72262915e-22
  3.82982985e-14]]
